In [33]:
import sys
import os
import re
import subprocess
from pathlib import Path
import logging
from datetime import datetime


**Session Initialization & Configuration**

**What This Code Does:**
 

* **Navigate to project directory**

  * Compute  home folder (`HOME`) and define the project path (`AZURE_RAG_PROJECT`)
  * Change the current working directory to the project root
  * Print the active directory name

* **Ensure project is importable**

  * Check if the project path is already in `sys.path`
  * If not, append it so you can import modules from `azure-multimodal-rag`

* **Set up logging**

  * Build a timestamped log-file path under `logs/`
  * Configure the `logging` module to write INFO-level messages both to file and to the console
  * Print the log-file name and confirm the environment is ready

* **Load (or create) configuration**

  * Attempt to import `config` from your settings module
  * If that fails, define a fallback `AzureRAGConfig` class with sane defaults:

    * Raw PDF folder, processed data folder
    * Chunk size and overlap for text splitting
    * Supported file extensions
  * Instantiate `config` and confirm

* **Display current configuration**

  * Print out the active chunk size, overlap, and PDF folder path

* **Discover available PDFs**

  * Glob all `.pdf` files in `config.PDF_FOLDER`
  * Print the number of PDFs found and list each with its file size (in MB)

 

In [34]:

# Navigate to project directory
HOME = Path.home()
project_name = "azure-multimodal-rag"
AZURE_RAG_PROJECT = HOME / "projects" / project_name
os.chdir(AZURE_RAG_PROJECT)

print(f"📁 Working Directory: {AZURE_RAG_PROJECT.name}")

# Add to Python path
if str(AZURE_RAG_PROJECT) not in sys.path:
    sys.path.append(str(AZURE_RAG_PROJECT))

# Setup logging for this session
log_file = Path("logs") / f"doc_processing_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

print(f"📋 Session logged to: {log_file.name}")
print("✅ Environment ready!")

# Load config (recreate if needed)
try:
    from config.settings import config
    print(f"✅ Configuration loaded from file")
except ImportError:
    print("⚠️  Creating config...")
    class AzureRAGConfig:
        PDF_FOLDER = "data/raw/pdfs"
        PROCESSED_FOLDER = "data/processed"
        CHUNK_SIZE = 1000
        CHUNK_OVERLAP = 200
        SUPPORTED_FILE_TYPES = [".pdf", ".txt", ".md"]
    
    config = AzureRAGConfig()
    print(f"✅ Configuration ready")

print(f"\n📊 Current Config:")
print(f"   📏 Chunk Size: {config.CHUNK_SIZE}")
print(f"   🔄 Overlap: {config.CHUNK_OVERLAP}")
print(f"   📁 PDF Folder: {config.PDF_FOLDER}")

# Check what PDFs we have
pdf_files = list(Path(config.PDF_FOLDER).glob("*.pdf"))
print(f"\n📄 Available PDFs: {len(pdf_files)}")
for pdf in pdf_files:
    size_mb = pdf.stat().st_size / (1024 * 1024)
    print(f"   📄 {pdf.name} ({size_mb:.1f} MB)")


📁 Working Directory: azure-multimodal-rag
📋 Session logged to: doc_processing_20250629_143210.log
✅ Environment ready!
✅ Configuration loaded from file

📊 Current Config:
   📏 Chunk Size: 1000
   🔄 Overlap: 200
   📁 PDF Folder: data/raw/pdfs

📄 Available PDFs: 2
   📄 01-study-guide-az-vnet.pdf (0.1 MB)
   📄 02-study-guide-az-load-balancer.pdf (0.2 MB)



**PDF Extraction with Metadata & Structure Analysis**

**What This Code Does:**
  
* **Initialization (`__init__`)**

  * Stores the passed-in `config` object for folder paths and settings.
  * Configures a class-specific logger (e.g. `__name__.EnhancedPDFReader`) at INFO level.

* **`read_pdf_with_metadata(pdf_path)`**

  1. **Imports & Validation**:

     * Tries to import PyMuPDF (`fitz`) and raises a clear error if missing.
  2. **Timing & Logging**:

     * Records start time and logs the PDF filename being processed.
  3. **Metadata Extraction**:

     * Calls `_extract_document_metadata` to pull file stats (size, timestamps) and embedded PDF metadata (title, author, etc.).
  4. **Page-Level Content**:

     * Invokes `_extract_pages_content` to loop through each page—capturing text, char/word/line counts, densities, and logs progress every 10 pages.
  5. **Aggregate Statistics**:

     * Merges all page text, then uses `_calculate_content_statistics` to derive totals (chars, words, unique terms), distributions (alphabetic, numeric, punctuation), density ratios, and a simple readability score.
  6. **Structure Detection**:

     * Runs `_detect_structure_patterns` to spot headings, bullets, code blocks, tables, URLs, and Azure-specific naming patterns, yielding an estimated section count and a “structure confidence” score.
  7. **Result Assembly**:

     * Closes the document, measures elapsed time, logs a summary line, and returns a dictionary containing:

       * `content` (full text)
       * `metadata`, `pages`, `content_stats`, `structure_hints`
       * `processing_info` (timing, timestamp, version)

* **Helper Methods**

  * **`_extract_document_metadata`**: Gathers filesystem info (size, modified time) and PDF properties (creator, producer, dates).
  * **`_extract_pages_content`**: Iterates pages to get raw text, counts, densities, and logs progress.
  * **`_calculate_content_statistics`**: Computes global counts, char-type breakdown, page-level aggregates, density ratios, and calls `_estimate_readability`.
  * **`_detect_structure_patterns`**: Uses regex to count section numbers, bullets, ALL-CAP lines, title-case lines, code/tables/URLs, and Azure tags.
  * **`_estimate_readability`**: Approximates Flesch Reading Ease via syllable counting and sentence lengths.
  * **`_calculate_structure_confidence`**: Scores the presence of structural cues (numbered sections, bullets, tables, code) to a 0–1 confidence value.

* **Test Block**

  * Locates the first PDF in `config.PDF_FOLDER`.
  * Instantiates `EnhancedPDFReader` and calls `read_pdf_with_metadata`.
  * Prints out key results: title, pages, file size, total chars/words/unique words, readability score, structure confidence, and processing time.
  * Summarizes structure hints (estimated sections, bullets, URLs, Azure services).



In [35]:
# Cell 3: Document Quality Assessor
print("🔍 Building Document Quality Assessor...")

from enum import Enum
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import re

class ProcessingTier(Enum):
    """Processing tiers based on document quality"""
    PREMIUM = "premium"
    STANDARD = "standard" 
    BASIC = "basic"
    MANUAL_REVIEW = "manual_review"
    SKIP = "skip"

class ReviewType(Enum):
    """Types of human review needed"""
    NONE = "none"
    SPOT_CHECK = "spot_check"
    CHUNKING_VALIDATION = "chunking_validation"
    FULL_STRUCTURAL_REVIEW = "full_structural_review"
    OCR_CLEANUP = "ocr_cleanup"

@dataclass
class QualityAssessment:
    """Comprehensive document quality assessment result"""
    overall_score: float
    processing_tier: ProcessingTier
    review_type: ReviewType
    chunking_strategy: str
    estimated_processing_cost: float
    confidence_factors: Dict
    recommendations: List[str]
    should_process: bool
    priority_score: int

class DocumentQualityAssessor:
    """Production-ready document quality assessment system"""
    
    def __init__(self, config):
        self.config = config
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        
        # Quality thresholds for different processing decisions
        self.thresholds = {
            'premium_processing': 0.8,      # Top-tier processing
            'standard_processing': 0.6,     # Standard automation
            'basic_processing': 0.4,        # Minimal processing
            'manual_review': 0.2,           # Human review required
            'skip_processing': 0.1          # Don't process
        }
        
        # Cost estimates (in processing units)
        self.processing_costs = {
            ProcessingTier.PREMIUM: 10.0,
            ProcessingTier.STANDARD: 5.0,
            ProcessingTier.BASIC: 2.0,
            ProcessingTier.MANUAL_REVIEW: 15.0,  # Includes human time
            ProcessingTier.SKIP: 0.1
        }
        
        self.logger.info("Document Quality Assessor initialized")
    
    def assess_document_quality(self, pdf_data: Dict) -> QualityAssessment:
        """Comprehensive quality assessment with processing recommendations"""
        
        self.logger.info(f"Assessing quality for: {pdf_data['metadata']['filename']}")
        
        # Individual quality components
        structure_quality = self._assess_structure_quality(pdf_data)
        content_quality = self._assess_content_quality(pdf_data)
        technical_quality = self._assess_technical_quality(pdf_data)
        ocr_quality = self._assess_ocr_quality(pdf_data)
        business_value = self._assess_business_value(pdf_data)
        
        # Confidence factors for transparency
        confidence_factors = {
            'structure_quality': structure_quality,
            'content_quality': content_quality,
            'technical_quality': technical_quality,
            'ocr_quality': ocr_quality,
            'business_value': business_value
        }
        
        # Calculate weighted overall score
        overall_score = self._calculate_overall_score(confidence_factors)
        
        # Determine processing strategy
        processing_tier = self._determine_processing_tier(overall_score, confidence_factors)
        review_type = self._determine_review_type(overall_score, confidence_factors)
        chunking_strategy = self._recommend_chunking_strategy(pdf_data, overall_score)
        
        # Cost-benefit analysis
        estimated_cost = self.processing_costs[processing_tier]
        should_process = self._should_process_document(overall_score, estimated_cost, business_value)
        
        # Generate recommendations
        recommendations = self._generate_recommendations(pdf_data, confidence_factors)
        
        # Calculate priority for processing queue
        priority_score = self._calculate_priority_score(overall_score, business_value, estimated_cost)
        
        assessment = QualityAssessment(
            overall_score=overall_score,
            processing_tier=processing_tier,
            review_type=review_type,
            chunking_strategy=chunking_strategy,
            estimated_processing_cost=estimated_cost,
            confidence_factors=confidence_factors,
            recommendations=recommendations,
            should_process=should_process,
            priority_score=priority_score
        )
        
        self.logger.info(f"Quality assessment complete: {overall_score:.2f} -> {processing_tier.value}")
        return assessment
    
    def _assess_structure_quality(self, pdf_data: Dict) -> float:
        """Assess document structure quality"""
        
        structure_hints = pdf_data['structure_hints']
        
        # Base score from structure confidence
        base_score = structure_hints['structure_confidence']
        
        # Bonus points for clear organization
        heading_score = 0.0
        headings = structure_hints['heading_patterns']
        
        if headings['numbered_sections'] > 0:
            heading_score += 0.3
        if headings['all_caps_lines'] > 0:
            heading_score += 0.2
        if headings['bullet_points'] > 0:
            heading_score += 0.1
        
        # Penalty for inconsistent structure
        if headings['numbered_sections'] > 0 and headings['bullet_points'] > headings['numbered_sections'] * 3:
            heading_score -= 0.1  # Too many bullets vs sections
        
        final_score = min(1.0, base_score + heading_score)
        
        self.logger.debug(f"Structure quality: {final_score:.2f} (base: {base_score:.2f}, heading: {heading_score:.2f})")
        return final_score
    
    def _assess_content_quality(self, pdf_data: Dict) -> float:
        """Assess content quality and richness"""
        
        content_stats = pdf_data['content_stats']
        
        # Content richness indicators
        word_diversity = content_stats['unique_words'] / max(1, content_stats['total_words'])
        content_density = content_stats['content_density']
        
        # Check for meaningful content
        if content_stats['total_words'] < 100:
            return 0.1  # Too little content
        
        # Readability assessment (context-aware)
        readability = content_stats['readability_estimate']
        
        # For technical documents, adjust readability expectations
        if self._is_technical_document(pdf_data):
            # Technical docs can have lower readability
            readability_score = max(0.5, readability / 100)
        else:
            readability_score = readability / 100
        
        # Combine factors
        quality_score = (
            word_diversity * 0.3 +
            content_density * 0.2 +
            readability_score * 0.3 +
            min(1.0, content_stats['total_words'] / 1000) * 0.2  # Length bonus up to 1000 words
        )
        
        self.logger.debug(f"Content quality: {quality_score:.2f} (diversity: {word_diversity:.2f}, density: {content_density:.2f})")
        return min(1.0, quality_score)
    
    def _assess_technical_quality(self, pdf_data: Dict) -> float:
        """Assess technical content quality for Azure documents"""
        
        structure_hints = pdf_data['structure_hints']
        content = pdf_data['content'].lower()
        
        # Azure-specific quality indicators
        azure_mentions = structure_hints['azure_patterns']['azure_services']
        
        # Technical terminology density
        technical_terms = [
            'configuration', 'deployment', 'management', 'security',
            'network', 'virtual', 'resource', 'service', 'endpoint',
            'policy', 'rule', 'group', 'account', 'subscription'
        ]
        
        technical_density = sum(content.count(term) for term in technical_terms) / max(1, len(content.split()))
        
        # Code and configuration indicators
        has_code = structure_hints['content_indicators']['has_code_blocks']
        has_urls = structure_hints['content_indicators']['has_urls'] > 0
        
        # Calculate technical quality
        technical_score = 0.0
        
        if azure_mentions > 10:  # Good Azure coverage
            technical_score += 0.4
        elif azure_mentions > 5:
            technical_score += 0.2
        
        if technical_density > 0.05:  # 5% technical terms
            technical_score += 0.3
        elif technical_density > 0.02:
            technical_score += 0.15
        
        if has_code:
            technical_score += 0.2
        
        if has_urls:
            technical_score += 0.1
        
        self.logger.debug(f"Technical quality: {technical_score:.2f} (Azure mentions: {azure_mentions}, tech density: {technical_density:.3f})")
        return min(1.0, technical_score)
    
    def _assess_ocr_quality(self, pdf_data: Dict) -> float:
        """Assess OCR quality and text extraction reliability"""
        
        content = pdf_data['content']
        
        if not content.strip():
            return 0.0
        
        # OCR quality indicators
        total_chars = len(content)
        
        # Check for OCR artifacts
        random_chars = len(re.findall(r'[^\w\s\.\,\!\?\-\(\)\:\;]', content))
        random_char_ratio = random_chars / max(1, total_chars)
        
        # Check for broken words (common OCR issue)
        words = content.split()
        short_fragments = len([w for w in words if len(w) == 1 and w.isalpha()])
        fragment_ratio = short_fragments / max(1, len(words))
        
        # Check for missing spaces (words run together)
        long_words = len([w for w in words if len(w) > 20])
        long_word_ratio = long_words / max(1, len(words))
        
        # Calculate OCR quality score
        ocr_score = 1.0
        ocr_score -= random_char_ratio * 2.0    # Penalize random characters
        ocr_score -= fragment_ratio * 1.5       # Penalize fragmentation
        ocr_score -= long_word_ratio * 1.0      # Penalize missing spaces
        
        ocr_score = max(0.0, ocr_score)
        
        self.logger.debug(f"OCR quality: {ocr_score:.2f} (random chars: {random_char_ratio:.3f}, fragments: {fragment_ratio:.3f})")
        return ocr_score
    
    def _assess_business_value(self, pdf_data: Dict) -> float:
        """Assess business value and priority of the document"""
        
        content = pdf_data['content'].lower()
        metadata = pdf_data['metadata']
        
        # Document freshness
        file_age_days = 0  # Could calculate from file metadata
        freshness_score = max(0.5, 1.0 - (file_age_days / 365))  # Decay over a year
        
        # Content value indicators
        value_keywords = [
            'guide', 'tutorial', 'documentation', 'best practices',
            'architecture', 'deployment', 'configuration', 'troubleshooting'
        ]
        
        value_score = sum(content.count(keyword) for keyword in value_keywords) / max(1, len(content.split()))
        value_score = min(1.0, value_score * 100)  # Scale up
        
        # Document size (comprehensive content is valuable)
        size_score = min(1.0, metadata['page_count'] / 20)  # Up to 20 pages gets full points
        
        # Combine factors
        business_value = (
            freshness_score * 0.3 +
            value_score * 0.4 +
            size_score * 0.3
        )
        
        self.logger.debug(f"Business value: {business_value:.2f} (freshness: {freshness_score:.2f}, value: {value_score:.2f})")
        return business_value
    
    def _calculate_overall_score(self, confidence_factors: Dict) -> float:
        """Calculate weighted overall quality score"""
        
        weights = {
            'structure_quality': 0.25,
            'content_quality': 0.25,
            'technical_quality': 0.20,
            'ocr_quality': 0.20,
            'business_value': 0.10
        }
        
        overall_score = sum(
            confidence_factors[factor] * weight 
            for factor, weight in weights.items()
        )
        
        return min(1.0, overall_score)
    
    def _determine_processing_tier(self, overall_score: float, confidence_factors: Dict) -> ProcessingTier:
        """Determine appropriate processing tier"""
        
        # Check for specific issues that override score
        if confidence_factors['ocr_quality'] < 0.3:
            return ProcessingTier.MANUAL_REVIEW
        
        # Score-based tiers
        if overall_score >= self.thresholds['premium_processing']:
            return ProcessingTier.PREMIUM
        elif overall_score >= self.thresholds['standard_processing']:
            return ProcessingTier.STANDARD
        elif overall_score >= self.thresholds['basic_processing']:
            return ProcessingTier.BASIC
        elif overall_score >= self.thresholds['manual_review']:
            return ProcessingTier.MANUAL_REVIEW
        else:
            return ProcessingTier.SKIP
    
    def _determine_review_type(self, overall_score: float, confidence_factors: Dict) -> ReviewType:
        """Determine type of human review needed"""
        
        if confidence_factors['ocr_quality'] < 0.5:
            return ReviewType.OCR_CLEANUP
        elif confidence_factors['structure_quality'] < 0.3:
            return ReviewType.FULL_STRUCTURAL_REVIEW
        elif overall_score < 0.6:
            return ReviewType.CHUNKING_VALIDATION
        elif overall_score >= 0.8:
            return ReviewType.NONE
        else:
            return ReviewType.SPOT_CHECK
    
    def _recommend_chunking_strategy(self, pdf_data: Dict, overall_score: float) -> str:
        """Recommend chunking strategy based on quality assessment"""
        
        structure_confidence = pdf_data['structure_hints']['structure_confidence']
        
        if overall_score >= 0.8 and structure_confidence >= 0.7:
            return "semantic_section_based"
        elif overall_score >= 0.6 and structure_confidence >= 0.5:
            return "smart_boundary_chunking"
        elif overall_score >= 0.4:
            return "sentence_boundary_chunking"
        else:
            return "fixed_size_chunking"
    
    def _should_process_document(self, overall_score: float, estimated_cost: float, business_value: float) -> bool:
        """Cost-benefit analysis for processing decision"""
        
        # Simple ROI calculation
        expected_benefit = overall_score * business_value * 10  # Scale factor
        roi = expected_benefit / max(0.1, estimated_cost)
        
        # Process if ROI > 1.0 and minimum quality threshold met
        return roi > 1.0 and overall_score > 0.2
    
    def _generate_recommendations(self, pdf_data: Dict, confidence_factors: Dict) -> List[str]:
        """Generate actionable recommendations"""
        
        recommendations = []
        
        if confidence_factors['structure_quality'] < 0.5:
            recommendations.append("Consider manual section marking for better structure detection")
        
        if confidence_factors['ocr_quality'] < 0.7:
            recommendations.append("OCR quality issues detected - consider reprocessing with better OCR")
        
        if confidence_factors['technical_quality'] < 0.3:
            recommendations.append("Low technical content density - verify document relevance")
        
        if pdf_data['content_stats']['total_words'] < 500:
            recommendations.append("Short document - consider combining with related documents")
        
        azure_services = pdf_data['structure_hints']['azure_patterns']['azure_services']
        if azure_services > 50:
            recommendations.append("High Azure service density - consider specialized Azure chunking")
        
        return recommendations
    
    def _calculate_priority_score(self, overall_score: float, business_value: float, estimated_cost: float) -> int:
        """Calculate priority score for processing queue (0-100)"""
        
        # Higher score = higher priority
        priority = (overall_score * 40 + business_value * 40 + (10 / max(1, estimated_cost)) * 20)
        return min(100, int(priority * 100))
    
    def _is_technical_document(self, pdf_data: Dict) -> bool:
        """Detect if document is technical in nature"""
        
        azure_mentions = pdf_data['structure_hints']['azure_patterns']['azure_services']
        technical_indicators = pdf_data['structure_hints']['content_indicators']
        
        return (
            azure_mentions > 5 or
            technical_indicators['has_code_blocks'] or
            technical_indicators['has_urls'] > 2
        )


# Test the Document Quality Assessor
print("\n🧪 Testing Document Quality Assessor...")

if 'pdf_data' in globals() and pdf_data:
    # Create quality assessor
    quality_assessor = DocumentQualityAssessor(config)
    
    # Assess our Azure document
    assessment = quality_assessor.assess_document_quality(pdf_data)
    
    print(f"\n📊 Quality Assessment Results:")
    print(f"   🎯 Overall Score: {assessment.overall_score:.2f}/1.0")
    print(f"   🏆 Processing Tier: {assessment.processing_tier.value.upper()}")
    print(f"   👥 Review Type: {assessment.review_type.value.replace('_', ' ').title()}")
    print(f"   🧩 Chunking Strategy: {assessment.chunking_strategy}")
    print(f"   💰 Estimated Cost: {assessment.estimated_processing_cost:.1f} units")
    print(f"   ✅ Should Process: {assessment.should_process}")
    print(f"   📈 Priority Score: {assessment.priority_score}/100")
    
    print(f"\n🔍 Quality Breakdown:")
    for factor, score in assessment.confidence_factors.items():
        print(f"   📊 {factor.replace('_', ' ').title()}: {score:.2f}")
    
    print(f"\n💡 Recommendations:")
    for i, rec in enumerate(assessment.recommendations, 1):
        print(f"   {i}. {rec}")
    
    print(f"\n✅ Document Quality Assessor working perfectly!")
    
else:
    print("❌ No PDF data available for assessment")

print(f"\n🎯 Ready for adaptive chunking strategies!")


# Cell 4: Batch Document Processing Pipeline
print("🏭 Building Batch Document Processing Pipeline...")

class BatchDocumentProcessor:
    """Process multiple documents with quality assessment and routing"""
    
    def __init__(self, config):
        self.config = config
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        
        # Initialize components
        self.pdf_reader = EnhancedPDFReader(config)
        self.quality_assessor = DocumentQualityAssessor(config)
        
        # Processing statistics
        self.stats = {
            'total_processed': 0,
            'premium_tier': 0,
            'standard_tier': 0,
            'basic_tier': 0,
            'manual_review': 0,
            'skipped': 0,
            'processing_errors': 0
        }
        
        self.logger.info("Batch Document Processor initialized")
    
    def process_all_pdfs(self, pdf_folder: str = None) -> List[Dict]:
        """Process all PDFs in the folder with quality assessment"""
        
        if pdf_folder is None:
            pdf_folder = self.config.PDF_FOLDER
        
        pdf_files = list(Path(pdf_folder).glob("*.pdf"))
        
        if not pdf_files:
            self.logger.warning(f"No PDF files found in {pdf_folder}")
            return []
        
        print(f"\n📄 Found {len(pdf_files)} PDF files to process:")
        for i, pdf_file in enumerate(pdf_files, 1):
            size_mb = pdf_file.stat().st_size / (1024 * 1024)
            print(f"   {i}. {pdf_file.name} ({size_mb:.1f} MB)")
        
        # Process each PDF
        results = []
        
        for pdf_file in pdf_files:
            try:
                result = self.process_single_pdf(pdf_file)
                results.append(result)
                self.stats['total_processed'] += 1
                
                # Update tier statistics
                tier = result['quality_assessment'].processing_tier.value
                if tier in self.stats:
                    self.stats[tier] += 1
                
            except Exception as e:
                self.logger.error(f"Error processing {pdf_file.name}: {str(e)}")
                self.stats['processing_errors'] += 1
                
                # Add error result
                results.append({
                    'filename': pdf_file.name,
                    'status': 'error',
                    'error': str(e),
                    'quality_assessment': None,
                    'processing_recommendation': 'manual_investigation'
                })
        
        # Print summary
        self._print_processing_summary(results)
        
        return results
    
    def process_single_pdf(self, pdf_path: Path) -> Dict:
        """Process a single PDF with quality assessment"""
        
        self.logger.info(f"Processing: {pdf_path.name}")
        
        # Extract PDF content and metadata
        pdf_data = self.pdf_reader.read_pdf_with_metadata(pdf_path)
        
        # Assess document quality
        assessment = self.quality_assessor.assess_document_quality(pdf_data)
        
        # Create processing result
        result = {
            'filename': pdf_path.name,
            'status': 'processed',
            'pdf_data': pdf_data,
            'quality_assessment': assessment,
            'processing_recommendation': self._get_processing_recommendation(assessment)
        }
        
        return result
    
    def _get_processing_recommendation(self, assessment: QualityAssessment) -> Dict:
        """Generate detailed processing recommendation"""
        
        recommendation = {
            'action': assessment.processing_tier.value,
            'priority': assessment.priority_score,
            'chunking_strategy': assessment.chunking_strategy,
            'estimated_cost': assessment.estimated_processing_cost,
            'should_process_immediately': assessment.should_process and assessment.processing_tier != ProcessingTier.MANUAL_REVIEW,
            'human_review_required': assessment.review_type != ReviewType.NONE,
            'review_type': assessment.review_type.value,
            'recommendations': assessment.recommendations
        }
        
        return recommendation
    
    def _print_processing_summary(self, results: List[Dict]) -> None:
        """Print comprehensive processing summary"""
        
        print(f"\n📊 Batch Processing Summary:")
        print(f"   📄 Total Documents: {len(results)}")
        print(f"   ✅ Successfully Processed: {self.stats['total_processed']}")
        print(f"   ❌ Processing Errors: {self.stats['processing_errors']}")
        
        print(f"\n🏆 Processing Tier Distribution:")
        total_successful = self.stats['total_processed']
        if total_successful > 0:
            print(f"   🥇 Premium: {self.stats.get('premium', 0)} ({self.stats.get('premium', 0)/total_successful*100:.1f}%)")
            print(f"   🥈 Standard: {self.stats.get('standard', 0)} ({self.stats.get('standard', 0)/total_successful*100:.1f}%)")
            print(f"   🥉 Basic: {self.stats.get('basic', 0)} ({self.stats.get('basic', 0)/total_successful*100:.1f}%)")
            print(f"   👥 Manual Review: {self.stats.get('manual_review', 0)} ({self.stats.get('manual_review', 0)/total_successful*100:.1f}%)")
            print(f"   ⏭️ Skipped: {self.stats.get('skip', 0)} ({self.stats.get('skip', 0)/total_successful*100:.1f}%)")
        
        # Document-level details
        print(f"\n📋 Document Details:")
        for i, result in enumerate(results, 1):
            if result['status'] == 'processed':
                assessment = result['quality_assessment']
                print(f"   {i}. {result['filename']}")
                print(f"      🎯 Score: {assessment.overall_score:.2f} | Tier: {assessment.processing_tier.value.upper()}")
                print(f"      🧩 Strategy: {assessment.chunking_strategy}")
                print(f"      💰 Cost: {assessment.estimated_processing_cost:.1f} units")
                if assessment.recommendations:
                    print(f"      💡 Rec: {assessment.recommendations[0]}")
            else:
                print(f"   {i}. {result['filename']} - ERROR: {result['error']}")
        
        # Cost analysis
        total_cost = sum(r['quality_assessment'].estimated_processing_cost 
                        for r in results if r['status'] == 'processed')
        
        print(f"\n💰 Cost Analysis:")
        print(f"   💸 Total Estimated Cost: {total_cost:.1f} processing units")
        print(f"   📈 Average Cost per Document: {total_cost/max(1, total_successful):.1f} units")
        
        # Recommendations summary
        all_recommendations = []
        for result in results:
            if result['status'] == 'processed':
                all_recommendations.extend(result['quality_assessment'].recommendations)
        
        if all_recommendations:
            from collections import Counter
            common_recs = Counter(all_recommendations).most_common(3)
            print(f"\n🔍 Most Common Recommendations:")
            for rec, count in common_recs:
                print(f"   • {rec} ({count} documents)")


# Test Batch Processing
print("\n🧪 Testing Batch Document Processing...")

# Create batch processor
batch_processor = BatchDocumentProcessor(config)

# Process all PDFs in the folder
batch_results = batch_processor.process_all_pdfs()

print(f"\n✅ Batch processing complete!")
print(f"🎯 Ready for adaptive chunking implementation!")

2025-06-29 14:32:10,860 - INFO - Document Quality Assessor initialized
2025-06-29 14:32:10,892 - INFO - Assessing quality for: 01-study-guide-az-vnet.pdf
2025-06-29 14:32:10,896 - INFO - Quality assessment complete: 0.83 -> premium
2025-06-29 14:32:10,898 - INFO - Enhanced PDF Reader initialized
2025-06-29 14:32:10,899 - INFO - Document Quality Assessor initialized
2025-06-29 14:32:10,900 - INFO - Batch Document Processor initialized
2025-06-29 14:32:10,921 - INFO - Processing: 01-study-guide-az-vnet.pdf
2025-06-29 14:32:10,922 - INFO - Processing PDF: 01-study-guide-az-vnet.pdf
2025-06-29 14:32:10,943 - INFO - Processed 10 pages...
2025-06-29 14:32:10,964 - INFO - PDF processed successfully: 25,984 chars, 11 pages, 0.04s
2025-06-29 14:32:10,975 - INFO - Assessing quality for: 01-study-guide-az-vnet.pdf
2025-06-29 14:32:10,978 - INFO - Quality assessment complete: 0.83 -> premium
2025-06-29 14:32:10,978 - INFO - Processing: 02-study-guide-az-load-balancer.pdf
2025-06-29 14:32:10,979 - 

🔍 Building Document Quality Assessor...

🧪 Testing Document Quality Assessor...

📊 Quality Assessment Results:
   🎯 Overall Score: 0.83/1.0
   🏆 Processing Tier: PREMIUM
   👥 Review Type: None
   🧩 Chunking Strategy: semantic_section_based
   💰 Estimated Cost: 10.0 units
   ✅ Should Process: False
   📈 Priority Score: 100/100

🔍 Quality Breakdown:
   📊 Structure Quality: 1.00
   📊 Content Quality: 0.61
   📊 Technical Quality: 0.90
   📊 Ocr Quality: 0.95
   📊 Business Value: 0.59

💡 Recommendations:
   1. High Azure service density - consider specialized Azure chunking

✅ Document Quality Assessor working perfectly!

🎯 Ready for adaptive chunking strategies!
🏭 Building Batch Document Processing Pipeline...

🧪 Testing Batch Document Processing...

📄 Found 2 PDF files to process:
   1. 01-study-guide-az-vnet.pdf (0.1 MB)
   2. 02-study-guide-az-load-balancer.pdf (0.2 MB)


2025-06-29 14:32:11,004 - INFO - Processed 10 pages...
2025-06-29 14:32:11,034 - INFO - Processed 20 pages...
2025-06-29 14:32:11,068 - INFO - PDF processed successfully: 55,835 chars, 25 pages, 0.09s
2025-06-29 14:32:11,069 - INFO - Assessing quality for: 02-study-guide-az-load-balancer.pdf
2025-06-29 14:32:11,074 - INFO - Quality assessment complete: 0.83 -> premium



📊 Batch Processing Summary:
   📄 Total Documents: 2
   ✅ Successfully Processed: 2
   ❌ Processing Errors: 0

🏆 Processing Tier Distribution:
   🥇 Premium: 0 (0.0%)
   🥈 Standard: 0 (0.0%)
   🥉 Basic: 0 (0.0%)
   👥 Manual Review: 0 (0.0%)
   ⏭️ Skipped: 0 (0.0%)

📋 Document Details:
   1. 01-study-guide-az-vnet.pdf
      🎯 Score: 0.83 | Tier: PREMIUM
      🧩 Strategy: semantic_section_based
      💰 Cost: 10.0 units
      💡 Rec: High Azure service density - consider specialized Azure chunking
   2. 02-study-guide-az-load-balancer.pdf
      🎯 Score: 0.83 | Tier: PREMIUM
      🧩 Strategy: semantic_section_based
      💰 Cost: 10.0 units
      💡 Rec: High Azure service density - consider specialized Azure chunking

💰 Cost Analysis:
   💸 Total Estimated Cost: 20.0 processing units
   📈 Average Cost per Document: 10.0 units

🔍 Most Common Recommendations:
   • High Azure service density - consider specialized Azure chunking (2 documents)

✅ Batch processing complete!
🎯 Ready for adaptive chu

In [36]:
# Cell 3: Document Quality Assessor
print("🔍 Building Document Quality Assessor...")

from enum import Enum
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import re

class ProcessingTier(Enum):
    """Processing tiers based on document quality"""
    PREMIUM = "premium"
    STANDARD = "standard" 
    BASIC = "basic"
    MANUAL_REVIEW = "manual_review"
    SKIP = "skip"

class ReviewType(Enum):
    """Types of human review needed"""
    NONE = "none"
    SPOT_CHECK = "spot_check"
    CHUNKING_VALIDATION = "chunking_validation"
    FULL_STRUCTURAL_REVIEW = "full_structural_review"
    OCR_CLEANUP = "ocr_cleanup"

@dataclass
class QualityAssessment:
    """Comprehensive document quality assessment result"""
    overall_score: float
    processing_tier: ProcessingTier
    review_type: ReviewType
    chunking_strategy: str
    estimated_processing_cost: float
    confidence_factors: Dict
    recommendations: List[str]
    should_process: bool
    priority_score: int

class DocumentQualityAssessor:
    """Production-ready document quality assessment system"""
    
    def __init__(self, config):
        self.config = config
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        
        # Quality thresholds for different processing decisions
        self.thresholds = {
            'premium_processing': 0.8,      # Top-tier processing
            'standard_processing': 0.6,     # Standard automation
            'basic_processing': 0.4,        # Minimal processing
            'manual_review': 0.2,           # Human review required
            'skip_processing': 0.1          # Don't process
        }
        
        # Cost estimates (in processing units)
        self.processing_costs = {
            ProcessingTier.PREMIUM: 10.0,
            ProcessingTier.STANDARD: 5.0,
            ProcessingTier.BASIC: 2.0,
            ProcessingTier.MANUAL_REVIEW: 15.0,  # Includes human time
            ProcessingTier.SKIP: 0.1
        }
        
        self.logger.info("Document Quality Assessor initialized")
    
    def assess_document_quality(self, pdf_data: Dict) -> QualityAssessment:
        """Comprehensive quality assessment with processing recommendations"""
        
        self.logger.info(f"Assessing quality for: {pdf_data['metadata']['filename']}")
        
        # Individual quality components
        structure_quality = self._assess_structure_quality(pdf_data)
        content_quality = self._assess_content_quality(pdf_data)
        technical_quality = self._assess_technical_quality(pdf_data)
        ocr_quality = self._assess_ocr_quality(pdf_data)
        business_value = self._assess_business_value(pdf_data)
        
        # Confidence factors for transparency
        confidence_factors = {
            'structure_quality': structure_quality,
            'content_quality': content_quality,
            'technical_quality': technical_quality,
            'ocr_quality': ocr_quality,
            'business_value': business_value
        }
        
        # Calculate weighted overall score
        overall_score = self._calculate_overall_score(confidence_factors)
        
        # Determine processing strategy
        processing_tier = self._determine_processing_tier(overall_score, confidence_factors)
        review_type = self._determine_review_type(overall_score, confidence_factors)
        chunking_strategy = self._recommend_chunking_strategy(pdf_data, overall_score)
        
        # Cost-benefit analysis
        estimated_cost = self.processing_costs[processing_tier]
        should_process = self._should_process_document(overall_score, estimated_cost, business_value)
        
        # Generate recommendations
        recommendations = self._generate_recommendations(pdf_data, confidence_factors)
        
        # Calculate priority for processing queue
        priority_score = self._calculate_priority_score(overall_score, business_value, estimated_cost)
        
        assessment = QualityAssessment(
            overall_score=overall_score,
            processing_tier=processing_tier,
            review_type=review_type,
            chunking_strategy=chunking_strategy,
            estimated_processing_cost=estimated_cost,
            confidence_factors=confidence_factors,
            recommendations=recommendations,
            should_process=should_process,
            priority_score=priority_score
        )
        
        self.logger.info(f"Quality assessment complete: {overall_score:.2f} -> {processing_tier.value}")
        return assessment
    
    def _assess_structure_quality(self, pdf_data: Dict) -> float:
        """Assess document structure quality"""
        
        structure_hints = pdf_data['structure_hints']
        
        # Base score from structure confidence
        base_score = structure_hints['structure_confidence']
        
        # Bonus points for clear organization
        heading_score = 0.0
        headings = structure_hints['heading_patterns']
        
        if headings['numbered_sections'] > 0:
            heading_score += 0.3
        if headings['all_caps_lines'] > 0:
            heading_score += 0.2
        if headings['bullet_points'] > 0:
            heading_score += 0.1
        
        # Penalty for inconsistent structure
        if headings['numbered_sections'] > 0 and headings['bullet_points'] > headings['numbered_sections'] * 3:
            heading_score -= 0.1  # Too many bullets vs sections
        
        final_score = min(1.0, base_score + heading_score)
        
        self.logger.debug(f"Structure quality: {final_score:.2f} (base: {base_score:.2f}, heading: {heading_score:.2f})")
        return final_score
    
    def _assess_content_quality(self, pdf_data: Dict) -> float:
        """Assess content quality and richness"""
        
        content_stats = pdf_data['content_stats']
        
        # Content richness indicators
        word_diversity = content_stats['unique_words'] / max(1, content_stats['total_words'])
        content_density = content_stats['content_density']
        
        # Check for meaningful content
        if content_stats['total_words'] < 100:
            return 0.1  # Too little content
        
        # Readability assessment (context-aware)
        readability = content_stats['readability_estimate']
        
        # For technical documents, adjust readability expectations
        if self._is_technical_document(pdf_data):
            # Technical docs can have lower readability
            readability_score = max(0.5, readability / 100)
        else:
            readability_score = readability / 100
        
        # Combine factors
        quality_score = (
            word_diversity * 0.3 +
            content_density * 0.2 +
            readability_score * 0.3 +
            min(1.0, content_stats['total_words'] / 1000) * 0.2  # Length bonus up to 1000 words
        )
        
        self.logger.debug(f"Content quality: {quality_score:.2f} (diversity: {word_diversity:.2f}, density: {content_density:.2f})")
        return min(1.0, quality_score)
    
    def _assess_technical_quality(self, pdf_data: Dict) -> float:
        """Assess technical content quality for Azure documents"""
        
        structure_hints = pdf_data['structure_hints']
        content = pdf_data['content'].lower()
        
        # Azure-specific quality indicators
        azure_mentions = structure_hints['azure_patterns']['azure_services']
        
        # Technical terminology density
        technical_terms = [
            'configuration', 'deployment', 'management', 'security',
            'network', 'virtual', 'resource', 'service', 'endpoint',
            'policy', 'rule', 'group', 'account', 'subscription'
        ]
        
        technical_density = sum(content.count(term) for term in technical_terms) / max(1, len(content.split()))
        
        # Code and configuration indicators
        has_code = structure_hints['content_indicators']['has_code_blocks']
        has_urls = structure_hints['content_indicators']['has_urls'] > 0
        
        # Calculate technical quality
        technical_score = 0.0
        
        if azure_mentions > 10:  # Good Azure coverage
            technical_score += 0.4
        elif azure_mentions > 5:
            technical_score += 0.2
        
        if technical_density > 0.05:  # 5% technical terms
            technical_score += 0.3
        elif technical_density > 0.02:
            technical_score += 0.15
        
        if has_code:
            technical_score += 0.2
        
        if has_urls:
            technical_score += 0.1
        
        self.logger.debug(f"Technical quality: {technical_score:.2f} (Azure mentions: {azure_mentions}, tech density: {technical_density:.3f})")
        return min(1.0, technical_score)
    
    def _assess_ocr_quality(self, pdf_data: Dict) -> float:
        """Assess OCR quality and text extraction reliability"""
        
        content = pdf_data['content']
        
        if not content.strip():
            return 0.0
        
        # OCR quality indicators
        total_chars = len(content)
        
        # Check for OCR artifacts
        random_chars = len(re.findall(r'[^\w\s\.\,\!\?\-\(\)\:\;]', content))
        random_char_ratio = random_chars / max(1, total_chars)
        
        # Check for broken words (common OCR issue)
        words = content.split()
        short_fragments = len([w for w in words if len(w) == 1 and w.isalpha()])
        fragment_ratio = short_fragments / max(1, len(words))
        
        # Check for missing spaces (words run together)
        long_words = len([w for w in words if len(w) > 20])
        long_word_ratio = long_words / max(1, len(words))
        
        # Calculate OCR quality score
        ocr_score = 1.0
        ocr_score -= random_char_ratio * 2.0    # Penalize random characters
        ocr_score -= fragment_ratio * 1.5       # Penalize fragmentation
        ocr_score -= long_word_ratio * 1.0      # Penalize missing spaces
        
        ocr_score = max(0.0, ocr_score)
        
        self.logger.debug(f"OCR quality: {ocr_score:.2f} (random chars: {random_char_ratio:.3f}, fragments: {fragment_ratio:.3f})")
        return ocr_score
    
    def _assess_business_value(self, pdf_data: Dict) -> float:
        """Assess business value and priority of the document"""
        
        content = pdf_data['content'].lower()
        metadata = pdf_data['metadata']
        
        # Document freshness
        file_age_days = 0  # Could calculate from file metadata
        freshness_score = max(0.5, 1.0 - (file_age_days / 365))  # Decay over a year
        
        # Content value indicators
        value_keywords = [
            'guide', 'tutorial', 'documentation', 'best practices',
            'architecture', 'deployment', 'configuration', 'troubleshooting'
        ]
        
        value_score = sum(content.count(keyword) for keyword in value_keywords) / max(1, len(content.split()))
        value_score = min(1.0, value_score * 100)  # Scale up
        
        # Document size (comprehensive content is valuable)
        size_score = min(1.0, metadata['page_count'] / 20)  # Up to 20 pages gets full points
        
        # Combine factors
        business_value = (
            freshness_score * 0.3 +
            value_score * 0.4 +
            size_score * 0.3
        )
        
        self.logger.debug(f"Business value: {business_value:.2f} (freshness: {freshness_score:.2f}, value: {value_score:.2f})")
        return business_value
    
    def _calculate_overall_score(self, confidence_factors: Dict) -> float:
        """Calculate weighted overall quality score"""
        
        weights = {
            'structure_quality': 0.25,
            'content_quality': 0.25,
            'technical_quality': 0.20,
            'ocr_quality': 0.20,
            'business_value': 0.10
        }
        
        overall_score = sum(
            confidence_factors[factor] * weight 
            for factor, weight in weights.items()
        )
        
        return min(1.0, overall_score)
    
    def _determine_processing_tier(self, overall_score: float, confidence_factors: Dict) -> ProcessingTier:
        """Determine appropriate processing tier"""
        
        # Check for specific issues that override score
        if confidence_factors['ocr_quality'] < 0.3:
            return ProcessingTier.MANUAL_REVIEW
        
        # Score-based tiers
        if overall_score >= self.thresholds['premium_processing']:
            return ProcessingTier.PREMIUM
        elif overall_score >= self.thresholds['standard_processing']:
            return ProcessingTier.STANDARD
        elif overall_score >= self.thresholds['basic_processing']:
            return ProcessingTier.BASIC
        elif overall_score >= self.thresholds['manual_review']:
            return ProcessingTier.MANUAL_REVIEW
        else:
            return ProcessingTier.SKIP
    
    def _determine_review_type(self, overall_score: float, confidence_factors: Dict) -> ReviewType:
        """Determine type of human review needed"""
        
        if confidence_factors['ocr_quality'] < 0.5:
            return ReviewType.OCR_CLEANUP
        elif confidence_factors['structure_quality'] < 0.3:
            return ReviewType.FULL_STRUCTURAL_REVIEW
        elif overall_score < 0.6:
            return ReviewType.CHUNKING_VALIDATION
        elif overall_score >= 0.8:
            return ReviewType.NONE
        else:
            return ReviewType.SPOT_CHECK
    
    def _recommend_chunking_strategy(self, pdf_data: Dict, overall_score: float) -> str:
        """Recommend chunking strategy based on quality assessment"""
        
        structure_confidence = pdf_data['structure_hints']['structure_confidence']
        
        if overall_score >= 0.8 and structure_confidence >= 0.7:
            return "semantic_section_based"
        elif overall_score >= 0.6 and structure_confidence >= 0.5:
            return "smart_boundary_chunking"
        elif overall_score >= 0.4:
            return "sentence_boundary_chunking"
        else:
            return "fixed_size_chunking"
    
    def _should_process_document(self, overall_score: float, estimated_cost: float, business_value: float) -> bool:
        """Cost-benefit analysis for processing decision"""
        
        # Simple ROI calculation
        expected_benefit = overall_score * business_value * 10  # Scale factor
        roi = expected_benefit / max(0.1, estimated_cost)
        
        # Process if ROI > 1.0 and minimum quality threshold met
        return roi > 1.0 and overall_score > 0.2
    
    def _generate_recommendations(self, pdf_data: Dict, confidence_factors: Dict) -> List[str]:
        """Generate actionable recommendations"""
        
        recommendations = []
        
        if confidence_factors['structure_quality'] < 0.5:
            recommendations.append("Consider manual section marking for better structure detection")
        
        if confidence_factors['ocr_quality'] < 0.7:
            recommendations.append("OCR quality issues detected - consider reprocessing with better OCR")
        
        if confidence_factors['technical_quality'] < 0.3:
            recommendations.append("Low technical content density - verify document relevance")
        
        if pdf_data['content_stats']['total_words'] < 500:
            recommendations.append("Short document - consider combining with related documents")
        
        azure_services = pdf_data['structure_hints']['azure_patterns']['azure_services']
        if azure_services > 50:
            recommendations.append("High Azure service density - consider specialized Azure chunking")
        
        return recommendations
    
    def _calculate_priority_score(self, overall_score: float, business_value: float, estimated_cost: float) -> int:
        """Calculate priority score for processing queue (0-100)"""
        
        # Higher score = higher priority
        priority = (overall_score * 40 + business_value * 40 + (10 / max(1, estimated_cost)) * 20)
        return min(100, int(priority * 100))
    
    def _is_technical_document(self, pdf_data: Dict) -> bool:
        """Detect if document is technical in nature"""
        
        azure_mentions = pdf_data['structure_hints']['azure_patterns']['azure_services']
        technical_indicators = pdf_data['structure_hints']['content_indicators']
        
        return (
            azure_mentions > 5 or
            technical_indicators['has_code_blocks'] or
            technical_indicators['has_urls'] > 2
        )


# Test the Document Quality Assessor
print("\n🧪 Testing Document Quality Assessor...")

if 'pdf_data' in globals() and pdf_data:
    # Create quality assessor
    quality_assessor = DocumentQualityAssessor(config)
    
    # Assess our Azure document
    assessment = quality_assessor.assess_document_quality(pdf_data)
    
    print(f"\n📊 Quality Assessment Results:")
    print(f"   🎯 Overall Score: {assessment.overall_score:.2f}/1.0")
    print(f"   🏆 Processing Tier: {assessment.processing_tier.value.upper()}")
    print(f"   👥 Review Type: {assessment.review_type.value.replace('_', ' ').title()}")
    print(f"   🧩 Chunking Strategy: {assessment.chunking_strategy}")
    print(f"   💰 Estimated Cost: {assessment.estimated_processing_cost:.1f} units")
    print(f"   ✅ Should Process: {assessment.should_process}")
    print(f"   📈 Priority Score: {assessment.priority_score}/100")
    
    print(f"\n🔍 Quality Breakdown:")
    for factor, score in assessment.confidence_factors.items():
        print(f"   📊 {factor.replace('_', ' ').title()}: {score:.2f}")
    
    print(f"\n💡 Recommendations:")
    for i, rec in enumerate(assessment.recommendations, 1):
        print(f"   {i}. {rec}")
    
    print(f"\n✅ Document Quality Assessor working perfectly!")
    
else:
    print("❌ No PDF data available for assessment")



2025-06-29 14:32:11,214 - INFO - Document Quality Assessor initialized
2025-06-29 14:32:11,215 - INFO - Assessing quality for: 01-study-guide-az-vnet.pdf
2025-06-29 14:32:11,219 - INFO - Quality assessment complete: 0.83 -> premium


🔍 Building Document Quality Assessor...

🧪 Testing Document Quality Assessor...

📊 Quality Assessment Results:
   🎯 Overall Score: 0.83/1.0
   🏆 Processing Tier: PREMIUM
   👥 Review Type: None
   🧩 Chunking Strategy: semantic_section_based
   💰 Estimated Cost: 10.0 units
   ✅ Should Process: False
   📈 Priority Score: 100/100

🔍 Quality Breakdown:
   📊 Structure Quality: 1.00
   📊 Content Quality: 0.61
   📊 Technical Quality: 0.90
   📊 Ocr Quality: 0.95
   📊 Business Value: 0.59

💡 Recommendations:
   1. High Azure service density - consider specialized Azure chunking

✅ Document Quality Assessor working perfectly!
